In [ ]:
import warnings
warnings.filterwarnings('ignore')
import flowermd
import hoomd
import gsd
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
from flowermd.base import Pack, Simulation
from flowermd.library import KremerGrestBeadSpring, LJChain
from flowermd.utils import get_target_box_number_density
from flowermd.base import Molecule
#from flowermd.library import Graphene
from flowermd.library.forcefields import BeadSpring
from flowermd.base.forcefield import BaseHOOMDForcefield
import unyt as u
import hoomd
import mbuild as mb
import numpy as np
from cmeutils.visualize import FresnelGSD
cpu = hoomd.device.CPU()

In [ ]:
del sim

In [ ]:
import mbuild as mb
from mbuild.compound import Compound
from mbuild.lattice import Lattice

from flowermd.base import Molecule, System


class Graphene(System):
    def __init__(
        self,
        x_repeat,
        y_repeat,
        n_layers,
        base_units=dict(),
        periodicity=(True, True, False),
    ):
        surface = mb.Compound(periodicity=periodicity)
        scale = 1.0 # CHANGE THIS TO SCALE!!
        spacings = [s * scale for s in [0.425, 0.246, 0.35]]
        points = [
            [1 / 6, 0, 0],
            [1 / 2, 0, 0],
            [0, 1 / 2, 0],
            [2 / 3, 1 / 2, 0],
        ]
        lattice = Lattice(
            lattice_spacing=spacings,
            angles=[90, 90, 90],
            lattice_points={"A": points},
        )
        carbon = Compound(name="C", element="C")
        layers = lattice.populate(
            compound_dict={"A": carbon}, x=x_repeat, y=y_repeat, z=n_layers
        )
        surface.add(layers)
        surface.freud_generate_bonds("C", "C", dmin=0.14*scale, dmax=0.145*scale)
        surface_mol = Molecule(num_mols=1, compound=surface)
        super(Graphene, self).__init__(
            molecules=[surface_mol],
            base_units=base_units,
        )

    def _build_system(self):
        return self.all_molecules[0]

In [ ]:
#OK, so we want to initialize a system with some chains and some flakes
kg_chain = LJChain(lengths=10,num_mols=1)
sheet = Graphene(x_repeat=3, y_repeat=3, n_layers=1, periodicity=(False,False,False))
#system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=1), kg_chain], density=0.2)

In [ ]:
system.hoomd_snapshot.configuration.box = [10, 10, 10, 0, 0, 0]

In [ ]:
ff = BeadSpring(r_cut=2.5,
        beads={"A": dict(epsilon=.1, sigma=1.0),
               "C": dict(epsilon=0.1, sigma=1.0)},
        bonds={"C-C": dict(r0=0.24599999029207287, k=50), "A-A": dict(r0=1.1, k=0.01)},
        angles={"A-A-A": dict(t0=np.pi, k=0.1),
                "C-C-C": dict(t0=2*np.pi/3., k=0.1)},
        dihedrals={"A-A-A-A": dict(phi0=0.0, k=0.1, d=-1, n=1), "C-C-C-C": dict(phi0=0.1, k=0.1, d=-1, n=1)})
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=cpu, dt = 0.0001)
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=0.1)
sim.run_NVT(n_steps=2e4,kT=2.0, tau_kt = 0.1) #short for initial testing
sim.flush_writers()
sim_visualizer = FresnelGSD(gsd_file="trajectory.gsd", frame=2, view_axis=(1, 1, 1))
sim_visualizer.view()
# dt of .0001 is as big as we can get with the current parameterization
#Issues:
# Graphene flakes are initialized way too small, or the bond lengths are wrong
# Need to double-check all the bonds, angle, dihedral parameters
# then need to go back and get some chain lengths and flake sizes right.